# Проект по оптимизации работы банкоматов
Этот проект предоставляет комплексное решение для анализа и оптимизации процессов обращения наличных в банкоматах. Решение включает реализацию всех этапов с использованием Python, с подробными шагами для выполнения задач.

## Этапы проекта

### Этап 1: Получение данных о ключевых ставках

Скрипт подключается к API Центрального банка России (ЦБ РФ) для получения данных о ключевых ставках за 2024 год. Используется протокол SOAP для извлечения данных.

In [10]:
import requests
from xml.etree import ElementTree
from datetime import datetime
import pandas as pd

def fetch_key_rates():
    url = "https://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx"
    headers = {"Content-Type": "text/xml; charset=utf-8"}
    body = """<?xml version="1.0" encoding="utf-8"?>
    <soap:Envelope xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
      <soap:Body>
        <KeyRate xmlns="http://web.cbr.ru/" />
      </soap:Body>
    </soap:Envelope>"""
    response = requests.post(url, data=body, headers=headers)

    if response.status_code != 200:
        raise Exception("Ошибка при получении данных о ключевых ставках.")

    tree = ElementTree.fromstring(response.content)
    rates = []
    for record in tree.findall(".//{http://web.cbr.ru/}KeyRateRecord"):
        date = datetime.strptime(record.find("{http://web.cbr.ru/}Date").text, "%Y-%m-%d")
        rate = float(record.find("{http://web.cbr.ru/}Value").text)
        if date.year == 2024:
            rates.append({"date": date, "rate": rate})

    return pd.DataFrame(rates)

In [11]:
# Получение данных
key_rates = fetch_key_rates()

### Этап 2: Расчёт упущенного процентного дохода

Рассчитывается доход, который мог быть получен при инвестировании средств вместо их хранения в банкоматах.

In [12]:
def calculate_lost_interest(transactions, key_rates):
    def get_rate_for_date(date):
        rate_row = key_rates[key_rates['date'] <= date].sort_values(by='date', ascending=False).head(1)
        return rate_row['rate'].iloc[0] if not rate_row.empty else 0

    transactions['lost_interest'] = transactions.apply(
        lambda row: row['bal_end_of_day'] * get_rate_for_date(row['date']) / 365, axis=1
    )
    return transactions

# Пример вызова
transactions = calculate_lost_interest(transactions, key_rates)

### Этап 3: Расчёт комиссий за инкассацию

Добавляется расчёт фиксированных и процентных комиссий для каждого банкомата.

In [5]:
def calculate_fees(transactions, fees):
    def calculate_fee(row):
        fee = fees.loc[row['atm_id']]
        cash_in = row['cash_in']
        cash_out = row['cash_out']
        delivery_fee = max(fee['CashDeliveryFixedFee'] + cash_in * fee['CashDeliveryPercentageFee'], fee['CashDeliveryMinFee'])
        collection_fee = max(fee['CashCollectionFixedFee'] + cash_out * fee['CashCollectionPercentageFee'], fee['CashCollectionMinFee'])
        return delivery_fee + collection_fee

    transactions['cash_handling_fee'] = transactions.apply(calculate_fee, axis=1)
    return transactions

# Пример вызова
# transactions = calculate_fees(transactions, fees)

### Этап 4: Оптимизация процессов

Простая модель для расчёта объёмов пополнения наличных с учётом уровня обслуживания 90%.

In [6]:
def optimize_cash_handling(transactions):
    transactions['optimized_cash_order'] = transactions['cash_out'] * 1.1
    return transactions

### Сохранение результатов

Результаты сохраняются в формате CSV для дальнейшего анализа:
# transactions.to_csv("optimized_transactions.csv", index=False)


## Инструкция по запуску

1. Установите зависимости:
   ```bash
   pip install pandas requests
   ```
2. Подготовьте данные:
   - Файл с транзакциями (`transactions.csv`)
   - Файл с комиссиями (`fees.csv`)

3. Запустите скрипты по этапам:
   - Получение ключевых ставок
   - Расчёт упущенного дохода и комиссий
   - Оптимизация заказов наличных

## Дальнейшая работа

- Разработка более сложной модели машинного обучения для предсказания объемов пополнений.
- Создание дашбордов для визуализации метрик.

## Контакты
Для вопросов и предложений обращайтесь на [sultanov55552@gmail.com].
